In [2]:
import pandas as pd

import plotly.express as px

from main import GameResult

In [9]:
def _sanity_check(x, CAT, animal_cols):
    cat_finished = x[CAT] == GameResult.FINISHED.name
    any_animal_finished = any(x[col] == GameResult.FINISHED.name for col in animal_cols)
    all_animals_chased = all(x[col] == GameResult.CHASED.name for col in animal_cols)
    assert cat_finished != any_animal_finished
    if any_animal_finished or all_animals_chased:
        assert any_animal_finished != all_animals_chased


def get_winner(x):
    CAT = "C"
    animal_cols = ["B", "S", "M"]
    _sanity_check(x, CAT, animal_cols)
    result = None
    for col in [CAT] + animal_cols:
        if x[col] == GameResult.FINISHED.name:
            if result is None:
                result = col
            else:
                raise ValueError(f"W? {result=}")
    return result

In [10]:
df = pd.read_csv("results.csv")
display(df.describe())
display(df.groupby(by="strategy").describe())

,epoch,position,game_result,shortcut_position,snacks_cnt,game_id,max_num_snacks,timestamp
count,2.323456e+06,2.323456e+06,2.323456e+06,1.742592e+06,580864.000000,2.323456e+06,2.323456e+06,2.323456e+06
mean,3.386402e+01,1.860397e+01,-4.063369e-01,2.100000e+01,-0.765558,5.507916e+03,3.014143e+00,1.748333e+12
std,2.513931e+01,1.302227e+01,6.141947e+00,8.164968e+00,3.893207,2.669394e+03,1.609090e+00,6.503353e+04
min,-1.000000e+00,0.000000e+00,-2.000000e+00,1.100000e+01,-39.000000,0.000000e+00,0.000000e+00,1.748333e+12
25%,1.400000e+01,7.000000e+00,-1.000000e+00,1.100000e+01,-1.000000,3.495000e+03,3.000000e+00,1.748333e+12
50%,3.000000e+01,1.600000e+01,-1.000000e+00,2.100000e+01,0.000000,5.848000e+03,3.000000e+00,1.748333e+12
75%,4.900000e+01,2.800000e+01,-1.000000e+00,3.100000e+01,1.000000,7.662000e+03,4.000000e+00,1.748333e+12
max,1.240000e+02,5.000000e+01,5.000000e+01,3.100000e+01,5.000000,9.726000e+03,5.000000e+00,1.748333e+12


epoch                                              \
                      count      mean        std  min   25%   50%   75%   
strategy                                                                  
CLOSEST_RUN_AWAY   831904.0  39.10746  27.615671 -1.0  16.0  35.0  58.0   
ONLY_ONE_RUN_AWAY  521140.0  22.21070  15.489034 -1.0   9.0  21.0  33.0   
RANDOM_SINGLE      970412.0  35.62716  25.118535 -1.0  15.0  32.0  52.0   

                          position             ... max_num_snacks       \
                     max     count       mean  ...            75%  max   
strategy                                       ...                       
CLOSEST_RUN_AWAY   124.0  831904.0  20.448426  ...            4.0  4.0   
ONLY_ONE_RUN_AWAY   81.0  521140.0  14.743505  ...            4.0  4.0   
RANDOM_SINGLE      124.0  970412.0  19.095962  ...            5.0  5.0   

                  timestamp                                            \
                      count          mean           std           min   
strategy                                                                
CLOSEST_RUN_AWAY   831904.0  1.748333e+12  56574.875819  1.748333e+12   
ONLY_ONE_RUN_AWAY  521140.0  1.748333e+12  62848.715208  1.748333e+12   
RANDOM_SINGLE      970412.0  1.748333e+12  72332.408482  1.748333e+12   

                                                                           
                            25%           50%           75%           max  
strategy                                                                   
CLOSEST_RUN_AWAY   1.748333e+12  1.748333e+12  1.748333e+12  1.748333e+12  
ONLY_ONE_RUN_AWAY  1.748333e+12  1.748333e+12  1.748333e+12  1.748333e+12  
RANDOM_SINGLE      1.748333e+12  1.748333e+12  1.748333e+12  1.748333e+12  

[3 rows x 64 columns]

In [11]:
ds = []
for g_id in df.game_id.unique():
    df_game = df[df["game_id"] == g_id]
    last_epoch = df_game.epoch.max()
    df_last_epoch = df_game[df_game["epoch"] == last_epoch]
    d = {"end_epoch": last_epoch, "game_id": g_id, "strategy": df_last_epoch["strategy"].iloc[0]}
    for idx, row in df_last_epoch.iterrows():
        d[row["name"]] = GameResult(row["game_result"]).name
    ds.append(d)

df_game_result = pd.DataFrame(ds)
df_game_result["winner"] = df_game_result.apply(get_winner, axis=1)

KeyboardInterrupt: 

In [ ]:
df_game_result[df_game_result["C"] != GameResult.FINISHED.name]
px.histogram(df_game_result, "winner", color="strategy")


: 

: 

In [ ]:
px.histogram(df_game_result, x="end_epoch", nbins=len(df_game_result.end_epoch.unique()), color="strategy")

: 

: 

: 

: 